In [17]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2
import math
import time

In [18]:
interpreter = tf.lite.Interpreter(model_path='MovenetLightning.tflite')
interpreter.allocate_tensors()

# Drawing of Keypoints and Connecting of Joints

In [19]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))

    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1)

In [20]:
EDGES = {
    (0, 1): 'm',  # Connects keypoints 0 (left shoulder) and 1 (right shoulder)
    (0, 2): 'c',  # Connects keypoints 0 (left shoulder) and 2 (left hip)
    (1, 3): 'm',  # Connects keypoints 1 (right shoulder) and 3 (right hip)
    (2, 4): 'c',  # Connects keypoints 2 (left hip) and 4 (left knee)
    (0, 5): 'm',  # Connects keypoints 0 (left shoulder) and 5 (left ankle)
    (0, 6): 'c',  # Connects keypoints 0 (left shoulder) and 6 (right hip)
    (5, 7): 'm',  # Connects keypoints 5 (left ankle) and 7 (left foot)
    (7, 9): 'm',  # Connects keypoints 7 (left foot) and 9 (left toe)
    (6, 8): 'c',  # Connects keypoints 6 (right hip) and 8 (right knee)
    (8, 10): 'c', # Connects keypoints 8 (right knee) and 10 (right ankle)
    (5, 6): 'y',  # Connects keypoints 5 (left ankle) and 6 (right hip)
    (5, 11): 'm', # Connects keypoints 5 (left ankle) and 11 (nose)
    (6, 12): 'c', # Connects keypoints 6 (right hip) and 12 (nose)
    (11, 12): 'y',# Connects keypoints 11 (nose) and 12 (nose)
    (11, 13): 'm',# Connects keypoints 11 (nose) and 13 (left eye)
    (13, 15): 'm',# Connects keypoints 13 (left eye) and 15 (left ear)
    (12, 14): 'c',# Connects keypoints 12 (nose) and 14 (right eye)
    (14, 16): 'c' # Connects keypoints 14 (right eye) and 16 (right ear)
}

HIGH_PLANK_EDGES = {
    (5, 11): 'm',   # Left shoulder to Left hip
    (11, 13): 'm',  # Left hip to Left knee
    (13, 15): 'm',  # Left knee to Left ankle
    (6, 12): 'c',   # Right shoulder to Right hip
    (12, 14): 'c',  # Right hip to Right knee
    (14, 16): 'c',  # Right knee to Right ankle
    (5, 7): 'm',    # Left shoulder to Left elbow
    (7, 9): 'm',    # Left elbow to Left wrist
    (6, 8): 'c',    # Right shoulder to Right elbow
    (8, 10): 'c'    # Right elbow to Right wrist
}



In [21]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))

    for edge, color in HIGH_PLANK_EDGES.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]

        if (c1 > confidence_threshold) & (c2 > confidence_threshold):
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [22]:
def all_keypoints_detected(keypoints, edges, confidence_threshold):
    num_keypoints = keypoints.shape[2]
    for edge, _ in edges.items():
        p1, p2 = edge
        c1 = keypoints[0][0][p1][2]  # Adjusted indexing to handle the extra dimension
        c2 = keypoints[0][0][p2][2]  # Adjusted indexing to handle the extra dimension
        if c1 < confidence_threshold or c2 < confidence_threshold:
            return False
    return True


In [23]:
# Function to calculate the angle between three points
def calculate_angle(p1, p2, p3):
    angle_radians = math.atan2(p3[1] - p2[1], p3[0] - p2[0]) - math.atan2(p1[1] - p2[1], p1[0] - p2[0])
    angle_degrees = math.degrees(angle_radians)
    return abs(angle_degrees)

# Function to check if the angle between three points is within a tolerance
def angle_within_tolerance(p1, p2, p3, target_angle, tolerance):
    angle = calculate_angle(p1, p2, p3)
    return abs(angle - target_angle) < tolerance

# Calculate Bird Dog

In [24]:
def is_bird_dog_correct(keypoints_with_scores, confidence_threshold):
    # Reshape keypoints to remove unnecessary dimensions
    keypoints = keypoints_with_scores[:, :2]  # Assuming we've flattened the array similar to the previous function

    # Check if all keypoints are detected with enough confidence
    if all(keypoints[:, 2] > confidence_threshold):
        # Define the keypoints for both sides
        left_wrist = keypoints[9][:2]
        right_wrist = keypoints[10][:2]
        left_shoulder = keypoints[5][:2]
        right_shoulder = keypoints[6][:2]
        left_hip = keypoints[11][:2]
        right_hip = keypoints[12][:2]
        left_knee = keypoints[13][:2]
        left_ankle = keypoints[15][:2]
        right_knee = keypoints[14][:2]
        right_ankle = keypoints[16][:2]

        # Determine which arm and leg are raised by checking the y-coordinates (the y-value increases downwards)
        if left_wrist[1] < right_wrist[1]:
            raised_arm = left_wrist
            raised_leg = right_knee
            raised_ankle = right_ankle
            supporting_leg = left_knee
            supporting_ankle = left_ankle
        else:
            raised_arm = right_wrist
            raised_leg = left_knee
            raised_ankle = left_ankle
            supporting_leg = right_knee
            supporting_ankle = right_ankle

        # Check if the raised arm is straight and aligned with the body
        is_raised_arm_straight = angle_within_tolerance(raised_arm, right_shoulder if raised_arm == right_wrist else left_shoulder, left_hip if raised_arm == left_wrist else right_hip, 180, 25)
        
        # Check if the raised leg is straight (hip -> knee -> ankle)
        is_raised_leg_straight = angle_within_tolerance(right_hip if raised_leg == right_knee else left_hip, raised_leg, raised_ankle, 180, 25)
        
        # Check if the unraised leg forms a right angle (hip -> knee -> ankle)
        is_supporting_leg_at_right_angle = angle_within_tolerance(right_hip if supporting_leg == right_knee else left_hip, supporting_leg, supporting_ankle, 90, 25)
        
        # If both raised arm and leg are straight, and the supporting leg is at a right angle, the pose is correct
        return is_raised_arm_straight and is_raised_leg_straight and is_supporting_leg_at_right_angle
    else:
        # If keypoints are not detected with high confidence, we cannot assert the pose is correct
        return False

# Calculate High Side Plank

In [25]:
def is_high_side_plank_correct(keypoints_with_scores, confidence_threshold):
    # Reshape keypoints to remove unnecessary dimensions
    keypoints = keypoints_with_scores[0, 0, :, :]

    # Check if all keypoints are detected with enough confidence
    if all(keypoints[:, 2] > confidence_threshold):
        # Define the keypoints for both sides
        left_wrist = keypoints[9][:2]
        left_elbow = keypoints[7][:2]
        left_shoulder = keypoints[5][:2]
        left_hip = keypoints[11][:2]
        left_knee = keypoints[13][:2]

        right_wrist = keypoints[10][:2]
        right_elbow = keypoints[8][:2]
        right_shoulder = keypoints[6][:2]
        right_hip = keypoints[12][:2]
        right_knee = keypoints[14][:2]

        # Check if the angles are close to 90 degrees with some tolerance
        is_left_arm_straight = angle_within_tolerance(left_wrist, left_elbow, left_shoulder, 180, 25)
        is_left_body_aligned = angle_within_tolerance(left_shoulder, left_hip, left_knee, 180, 25)
        
        is_right_arm_straight = angle_within_tolerance(right_wrist, right_elbow, right_shoulder, 180, 25)
        is_right_body_aligned = angle_within_tolerance(right_shoulder, right_hip, right_knee, 180, 25)
        
        return (is_left_arm_straight and is_left_body_aligned) or (is_right_arm_straight and is_right_body_aligned)
    else:
        # If keypoints are not detected with high confidence, we cannot assert the pose is correct
        return False


# Calculate Low Side Plank

In [26]:
def is_low_side_plank_correct(keypoints_with_scores, confidence_threshold):
    # Reshape keypoints to remove unnecessary dimensions
    keypoints = keypoints_with_scores[0, 0, :, :]

    # Check if all keypoints are detected with enough confidence
    if all(keypoints[:, 2] > confidence_threshold):
        # Define the keypoints for both sides
        left_wrist = keypoints[9][:2]
        left_elbow = keypoints[7][:2]
        left_shoulder = keypoints[5][:2]
        left_hip = keypoints[11][:2]
        left_knee = keypoints[13][:2]

        right_wrist = keypoints[10][:2]
        right_elbow = keypoints[8][:2]
        right_shoulder = keypoints[6][:2]
        right_hip = keypoints[12][:2]
        right_knee = keypoints[14][:2]
        
        # Calculate angles for both sides
        left_arm_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
        left_body_angle = calculate_angle(left_shoulder, left_hip, left_knee)
        
        right_arm_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
        right_body_angle = calculate_angle(right_shoulder, right_hip, right_knee)
        
        # Check if either side meets the criteria
        is_left_arm_straight = 70 <= left_arm_angle <= 110
        is_left_body_aligned = 130 <= left_body_angle <= 210
        
        is_right_arm_straight = 70 <= right_arm_angle <= 110
        is_right_body_aligned = 130 <= right_body_angle <= 210
        
        return (is_left_arm_straight and is_left_body_aligned) or (is_right_arm_straight and is_right_body_aligned)
    else:
        # If keypoints are not detected with high confidence, we cannot assert the pose is correct
        return False


# Calculate High Plank Angle

In [27]:
def calculate_high_plank_angles(keypoints):
    # Define the keypoints for both arms
    left_wrist = keypoints[9][:2]
    left_elbow = keypoints[7][:2]
    left_shoulder = keypoints[5][:2]

    right_wrist = keypoints[10][:2]
    right_elbow = keypoints[8][:2]
    right_shoulder = keypoints[6][:2]

    # Define the keypoints for the body
    left_hip = keypoints[11][:2]
    left_knee = keypoints[13][:2]
    
    right_hip = keypoints[12][:2]
    right_knee = keypoints[14][:2]

    # Calculate arm angles
    left_arm_angle = calculate_angle(left_wrist, left_elbow, left_shoulder)
    right_arm_angle = calculate_angle(right_wrist, right_elbow, right_shoulder)
    
    # Calculate body angles
    left_body_angle = calculate_angle(left_shoulder, left_hip, left_knee)
    right_body_angle = calculate_angle(right_shoulder, right_hip, right_knee)

    return left_arm_angle, right_arm_angle, left_body_angle, right_body_angle

def is_high_plank_correct(keypoints_with_scores, confidence_threshold):
    keypoints = keypoints_with_scores[0, 0, :, :]
    if all(keypoints[:, 2] > confidence_threshold):
        left_arm_angle, right_arm_angle, left_body_angle, right_body_angle = calculate_high_plank_angles(keypoints)

        # Check if both arms are nearly straight
        is_arms_straight = (160 <= left_arm_angle <= 180 and 160 <= right_arm_angle <= 180)
        # Check if the body is correctly aligned with a slight slant
        is_body_aligned = (130 <= left_body_angle <= 195 and 130 <= right_body_angle <= 195)
        
        return is_arms_straight and is_body_aligned
    else:
        return False


# Scoring Algorithm (NOT YET DONE)

# Live Video Camera Capture for Testing

In [28]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()

    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192, 192)
    input_image = tf.cast(img, dtype=tf.float32)

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])

    # Draw connections and key points
    draw_connections(frame, keypoints_with_scores, HIGH_PLANK_EDGES, 0.4) 
    draw_keypoints(frame, keypoints_with_scores, 0.4)

    # check if the pose is correct
    if is_high_side_plank_correct(keypoints_with_scores, 0.3):
        cv2.putText(frame, "Perfect Side Plank", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    elif is_high_plank_correct(keypoints_with_scores, 0.3):
        cv2.putText(frame, "Perfect High Plank", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    elif is_low_side_plank_correct(keypoints_with_scores, 0.3):
        cv2.putText(frame, "Perfect Low Plank", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    elif is_wall_sit_correct(keypoints_with_scores, 0.3):
        cv2.putText(frame, "Perfect Wall Sit", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('MoveNet Lightning', frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): # q to quit
        break

cap.release()
cv2.destroyAllWindows()